# Last left off
- how to give structured input to llm to improve accuracy
- few prompt strategy

# Context Relevance Prompt Experiment

In [1]:
import _global
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langsmith.evaluation import evaluate, evaluate_existing
from langsmith.schemas import Example, Run

## Experimental LLM Evaluator being tested

In [2]:
# Data model
class GradeDocuments(BaseModel):
    """ Pydantic object used to format LLM output
    * 0: irrelevant diagnosis \n
    * 1: correct diagnosis, but does not contain information to anser the user question \n
    * 2: correct diagnosis and contains information to answer the user question). \n    
    """
    score: int = Field(description="Documents grade based on correct diagnosis and relevant information")


#llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
llm = ChatOpenAI(model="gpt-4o", temperature=0)

structured_llm_grader = llm.with_structured_output(GradeDocuments)

In [6]:
system = """
You are a grader assessing the relevance of a retrieved document content to a query. \n
The query is a question about a medical diagnosis. \n
The document is a Python dictionary. The content of the document is under the "page_content" key in the dictionary. \n
Give a score for the document using the scoring system below. \n

# Scoring
- **0**: The content does not contain any information about the queried diagnosis \n
- **1**: The content contains information about the queried diagnosis, but the information does not answer the query \n
- **2**: The content contains information about the queried diagnosis and the information answers the query). \n
"""

human = """
# RETRIEVED DOCUMENT
{document} 

# QUERY
{query}
"""
prompt_gradedoc = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", human),
    ]
)

retrieval_grader = prompt_gradedoc | structured_llm_grader

def grade_docs(run, example) -> dict:
    """Grades all queries and corresponding documents in the db. used in eval type 1"""
    grade = retrieval_grader.invoke({"query": example.inputs["query"], "document": example.inputs["context"]})
    return {"key": "grade", "score": int(grade.score), "comment": "grade for doc"}

## Metrics for evaluating the LLM evaluator

In [7]:
# compare generated eval scores with ground truth eval scores
def avg_diff(runs: list[Run], examples: list[Example]) -> dict:
    score = 0
    for i, (run, example) in enumerate(zip(runs, examples)):
        score += run.feedback_stats["grade"]["avg"] - int(example.outputs["score"])

    score = score / len(runs)
    
    return {"key": "avg diff from true score", "score": score}


## Full pipeline for evaluating llm evaluator for context relevance

In [8]:
# generate eval scores
exp = evaluate(
    lambda x:x,
    data="prompt_test_GAS",
    evaluators=[grade_docs],
    experiment_prefix="Prompt_testing",
)

# generate eval score for the evaluator
evaluate_existing(exp.experiment_name, summary_evaluators=[avg_diff])


View the evaluation results for experiment: 'Prompt_testing-1528805c' at:
https://smith.langchain.com/o/edfbc8bb-c3a3-5c1e-8b48-11b5a8cfd8ac/datasets/4c17fe49-6797-40aa-a5fc-36f4809034f5/compare?selectedSessions=2860e1db-aa6d-4b0b-8de1-76d3f8ade962




0it [00:00, ?it/s]

View the evaluation results for experiment: 'Prompt_testing-1528805c' at:
https://smith.langchain.com/o/edfbc8bb-c3a3-5c1e-8b48-11b5a8cfd8ac/datasets/4c17fe49-6797-40aa-a5fc-36f4809034f5/compare?selectedSessions=2860e1db-aa6d-4b0b-8de1-76d3f8ade962




0it [00:00, ?it/s]

<ExperimentResults Prompt_testing-1528805c>

# Ground Truth Prompt Experiment 

## Experimental LLM Evaluator being tested

In [ ]:
class GroundTruth(BaseModel):
    """List facts in handout not based on ground truth"""
    list_of_false: List[str] = Field(description="List of facts in handout not based on ground truth")

# LLM with function call 
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm_grader = llm.with_structured_output(GroundTruth)

# Prompt 
system = """
You are an expert assessor tasked with evaluating whether the generated text (provided by the user between the XML tags GENERATED TEXT) is factually based on the provided context (provided by the user between the XML tags CONTEXT).  

Follow these steps:
    Step 1: Read the provided context carefully. Understand the information presented in the context.
    Step 2: Analyze each sentence in the GENERATED TEXT. Compare it with the provided CONTEXT to determine its factual basis. Sentences in the GENERATED TEXT that are similar (but not verbatim) to the sentences in provided CONTEXT, but are still factually aligned are considered factually based on the CONTEXT.
    Step 3: Identify sentences in the GENERATED TEXT that are not factually based on the context (not factually supported by the provided CONTEXT or directly contradicts the provided CONTEXT.) 

"""

human = """
<GENERATED TEXT>
{handout} 
</GENERATED TEXT>

<CONTEXT>
{contexts}
</CONTEXT>
"""

prompt_groundtruth = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", human),
    ]
)

grader_groundtruth = prompt_groundtruth | structured_llm_grader

def grade_groundtruth(run, example):
    try: # try to get outputs from run, otherwise it is from dataset
        handout = run.outputs["handout"]
        contexts = run.outputs["contexts"]
    except KeyError:
        handout = example.outputs["handout"]
        contexts = example.inputs["contexts"]
        
    result = grader_groundtruth.invoke({"handout": handout, "contexts": contexts}).list_of_false
    count = len(result)
    
    return {
        "key": "count", "score": count, "comment": "num infactual sentences",
        "sentences": result
    }


## Metrics for evaluating the LLM evaluator

In [10]:
# average difference of generated count vs ground truth count
def avg_diff_count(runs: list[Run], examples: list[Example]) -> dict:
    score = 0
    for i, (run, example) in enumerate(zip(runs, examples)):
        score += run.feedback_stats["count"]["avg"] - int(example.outputs["count"])

    score = score / len(runs)
    
    return {"key": "avg diff from true count", "score": score}



## Full pipeline for evaluating llm evaluator for context relevance

In [11]:
# generate eval scores
exp = evaluate(
    lambda x:x,
    data="Prompt_test_groundtruth",
    evaluators=[grade_groundtruth],
    experiment_prefix="Prompt_testing",
)


# generate eval score for the evaluator
evaluate_existing(exp.experiment_name, summary_evaluators=[avg_diff_count])


NameError: name 'grade_groundtruth' is not defined